In [1]:
import spacy
nlp = spacy.load('en_core_web_lg')

/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Customize pattern
pattern_ls = [{"pattern": [[{"LOWER": "charter"}, {"LOWER": "plus"}]], "attr": {"TAG": "NN", "POS": "NOUN"}}]

In [16]:
pattern_ls[0]['pattern'][0]

[{'LOWER': 'charter'}, {'LOWER': 'plus'}]

In [17]:
# Input text
text = "You are the charter plus member."

In [18]:
# initialize ruler
ruler = nlp.get_pipe('attribute_ruler')

In [19]:
for i in pattern_ls:
    index_len = len(i['pattern'][0])
    for j in range(index_len):
        ruler.add(patterns = i['pattern'], attrs = i['attr'], index = j)

In [20]:
doc = nlp(text)

In [21]:
print(doc[3].tag_, doc[3].pos_)
print(doc[4].tag_, doc[4].pos_)

NN NOUN
NN NOUN
